In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
import time

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

# Description of the Problem

Kickstarter success rate. 


Training and Testing error. Training time. ROC curve



2. the training and testing error rates you obtained running the various learning algorithms on your problems. At the very least you should include graphs that show performance on both training and test data as a function of training size (note that this implies that you need to design a classification problem that has more than a trivial amount of data) and--for the algorithms that are iterative--training times/iterations. Both of these kinds of graphs are referred to as learning curves, BTW.

3. analyses of your results. Why did you get the results you did? Compare and contrast the different algorithms. What sort of changes might you make to each of those algorithms to improve performance? How fast were they in terms of wall clock time? Iterations? Would cross validation help (and if it would, why didn't you implement it?)? How much performance was due to the problems you chose? How about the values you chose for learning rates, stopping criteria, pruning methods, and so forth (and why doesn't your analysis show results for the different values you chose?)? Which algorithm performed best? How do you define best? Be creative and think of as many questions you can, and as many answers as you can.


-Why did you get the results you did? 

-Compare and contrast the different algorithms. 

-What sort of changes might you make to each of those algorithms to improve performance? 

-How fast were they in terms of wall clock time? Iterations? 

-Would cross validation help (and if it would, why didn't you implement it?)? How much performance was due to the 
problems you chose? 

-How about the values you chose for learning rates, stopping criteria, pruning methods, and so forth (and why doesn't your analysis show results for the different values you chose?)? 

-Which algorithm performed best? How do you define best? Be creative and think of as many questions you can, and as many answers as you can.


In [99]:
df = pd.read_csv('BTCUSDKraken.csv', index_col=0, usecols=['Date','Volume (BTC)', 'Weighted Price'])
df = df.drop('2018-01-12')
df = df.replace([0], np.nan)
df = df.ffill(axis=None, inplace=False, limit=None, downcast=None)
lookback = 3

In [103]:
delta = df['Weighted Price'].diff()
dUp, dDown = delta.copy(), delta.copy()
dUp[dUp < 0] = 0
dDown[dDown > 0] = 0

RolUp = pd.rolling_mean(dUp, lookback)
RolDown = pd.rolling_mean(dDown, lookback).abs()

df['RSI'] = RolUp / RolDown

/Users/bryanbaek/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
  
/Users/bryanbaek/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
  import sys


In [105]:
sma = pd.rolling_mean(df['Weighted Price'], window = lookback, min_periods = lookback)
ema = pd.ewma(df['Weighted Price'], span = lookback, min_periods = lookback)
rolling_std = pd.rolling_std(df['Weighted Price'], window = lookback, min_periods = lookback)
up = (rolling_std * 2) + ema
down = (rolling_std * -2) + ema

/Users/bryanbaek/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=3,window=3,center=False).mean()
  """Entry point for launching an IPython kernel.
/Users/bryanbaek/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,span=3,min_periods=3,adjust=True).mean()
  
/Users/bryanbaek/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=3,window=3,center=False).std()
  This is separate from the ipykernel package so we can avoid doing imports until


In [106]:
#df['Price/SMA'] = df['Weighted Price']/sma
df['bbp'] = (df['Weighted Price'] - down) / (up - down)
df['Price/EMA'] = df['Weighted Price'] / ema
df['price_momentum'] = df['Weighted Price'].pct_change(periods = lookback - 1)
df['vol_momentum'] = df['Volume (BTC)'].pct_change(periods = lookback - 1)
df['bestaction'] = df['Weighted Price'].pct_change(periods = 1)

In descending order from the Upper Band:

%B Above 1 = Price is Above the Upper Band

%B Equal to 1 = Price is at the Upper Band

%B Above .50 = Price is Above the Middle Line

%B Below .50 = Price is Below the Middle Line

%B Equal to 0 = Price is at the Lower Band

%B Below 0 = Price is Below the Lower Band

Generally speaking .80 and .20 are also relevant levels.

%B Above .80 = Price is Nearing the Upper Band
%B Below .20 = Price is Nearing the Lower Band

In [107]:
suggestion_conditions = [
    (df['Price/EMA'] < 0.97) & (df['bbp'] < 0.25),
    (df['Price/EMA'] > 1.01) & (df['bbp'] > 0.75)]

actual_conditions = [
    (df['bestaction'] > 0), (df['bestaction'] < 0) ]

choices = [1, -1]
#df['Suggestion'] = np.select(suggestion_conditions, choices, default=0)
df['Action'] = np.select(actual_conditions, choices, default=0)

In [108]:
df = df.drop(['Volume (BTC)', 'Weighted Price', 'bestaction'], axis=1)
df = df.drop(df.index[0:lookback])

In [109]:
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how='any')

In [111]:
features = list(df.columns[0:-1])
X = df[features]
Y = df['Action']

#Shuffle = False b/c gotta train on earlier data and test on later data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, shuffle=False)

# 1. Decision Tree

In [140]:
DT = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=5, 
                                  min_samples_leaf=2, min_weight_fraction_leaf=0.0, max_features=None, 
                                  random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                  min_impurity_split=None, class_weight=None, presort=False)
#DT = DT.fit(X_train, Y_train)
#Ypreds = DT.predict(X_test)

In [141]:
#The least populated class in y has only 8 members, which is too few. 
#The minimum number of members in any class cannot be less than n_splits=9.
scores = cross_val_score(DT, X_train, Y_train, cv = 8)

print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.84        0.87878788  0.76767677  0.7979798   0.80808081  0.80808081
  0.85714286  0.82653061]
Accuracy: 0.82 (+/- 0.07)


Pruning:

min_samples_split : int, float, optional (default=2)
The minimum number of samples required to split an internal node

min_samples_leaf : int, float, optional (default=1)
The minimum number of samples required to be at a leaf node:

# 2. Neural Networks (Multi-layer Perceptron (MLP) )

Pro:
*Capability to learn non-linear models.

Con: 
*MLP with hidden layers have a non-convex loss function where there exists more than one local minimum. Therefore different random weight initializations can lead to different validation accuracy.

*MLP requires tuning a number of hyperparameters such as the number of hidden neurons, layers, and iterations.

*MLP is sensitive to feature scaling.


## Features:
1. hidden_layer_sizes : tuple, length = n_layers - 2, default (100,)

The ith element represents the number of neurons in the ith hidden layer.
hidden_layer_sizes=(10, 2) : 10 hidden layer with 2 hidden units

2. activation : {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default ‘relu’
Activation function for the hidden layer.

3. solver : {‘lbfgs’, ‘sgd’, ‘adam’}, default ‘adam’
The solver for weight optimization.

4. learning_rate : {‘constant’, ‘invscaling’, ‘adaptive’}, default ‘constant’
Learning rate schedule for weight updates.

In [131]:
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(hidden_layer_sizes=(20, 2), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', 
 learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
 random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, 
 early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)


#NN = NN.fit(X_train, Y_train) 
#Ypreds = NN.predict(X_test)

In [137]:
scores = cross_val_score(NN, X_train, Y_train, cv = 10)

print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.5625      0.425       0.5625      0.0125      0.56962025  0.58227848
  0.83544304  0.60759494  0.57692308  0.42857143]
Accuracy: 0.52 (+/- 0.40)


# 3. Boosting (Random Forest)

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and use averaging to improve the predictive accuracy and control over-fitting.

The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement if bootstrap=True (default).

Pros:

* It runs efficiently on large data bases.
* It can handle thousands of input variables without variable deletion.
* It gives estimates of what variables are important in the classification.
* It generates an internal unbiased estimate of the generalization error as the forest building progresses.
* It has an effective method for estimating missing data and maintains accuracy when a large proportion of the data are missing.
* It has methods for balancing error in class population unbalanced data sets.
* Generated forests can be saved for future use on other data.
* Prototypes are computed that give information about the relation between the variables and the classification.
* It computes proximities between pairs of cases that can be used in clustering, locating outliers, or (by scaling) give interesting views of the data.
* The capabilities of the above can be extended to unlabeled data, leading to unsupervised clustering, data views and outlier detection.
* It offers an experimental method for detecting variable interactions.
Number of Trees in RF
* Random forests does not overfit. You can run as many trees as you want. * It is fast. Running on a data set with 50,000 cases and 100 variables, it produced 100 trees in 11 minutes on a 800Mhz machine. For large data sets the major memory requirement is the storage of the data itself, and three integer arrays with the same dimensions as the data. If proximities are calculated, storage requirements grow as the number of cases times the number of trees.

* In random forests, there is no need for cross-validation or a separate test set to get an unbiased estimate of the test set error. It is estimated internally, during the run, as follows:

* Each tree is constructed using a different bootstrap sample from the original data. About one-third of the cases are left out of the bootstrap sample and not used in the construction of the kth tree.

https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#remarks
The RandomForestClassifier is trained using bootstrap aggregation, where each new tree is fit from a bootstrap sample of the training observations z_i = (x_i, y_i).

The out-of-bag (OOB) error is the average error for each z_i calculated using predictions from the trees that do not contain z_i in their respective bootstrap sample.

This allows the RandomForestClassifier to be fit and validated whilst being trained.

T. Hastie, R. Tibshirani and J. Friedman, “Elements of Statistical Learning Ed. 2”, p592-593, Springer, 2009.

## Features:

1. n_estimators : integer, optional (default=10)
The number of trees in the forest.

2. criterion : string, optional (default=”gini”)
The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 

3. max_features : int, float, string or None, optional (default=”auto”)
The number of features to consider when looking for the best split:

4. max_depth : integer or None, optional (default=None)
The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

5. min_samples_split : int, float, optional (default=2)
The minimum number of samples required to split an internal node

6. min_samples_leaf : int, float, optional (default=1)
The minimum number of samples required to be at a leaf node

In [139]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=10, criterion='gini', 
                            max_depth=None, min_samples_split=2, 
                            min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features='auto', max_leaf_nodes=None, 
                            min_impurity_decrease=0.0, min_impurity_split=None, 
                            bootstrap=True, oob_score=False, n_jobs=1, 
                            random_state=None, verbose=0, 
                            warm_start=False, class_weight=None)

#RF = RF.fit(X_train, Y_train) 
#Ypreds = RF.predict(X_test)

scores = cross_val_score(RF, X_train, Y_train, cv = 10)

print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.875       0.8625      0.85        0.825       0.78481013  0.86075949
  0.87341772  0.83544304  0.83333333  0.81818182]
Accuracy: 0.84 (+/- 0.05)


# 4. SVM

-at least 2 kernel functions

Pros: 
* Effective in high dimensional spaces.
* Still effective in cases where number of dimensions is greater than the number of samples.
* Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
* Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

Cons:

* If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.
* SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation

## Features:

1. Kernel(Linear, RBF, poly, sigmoid, precomputed)

Linear kernel = Straight Line (hyperplane) as the decision boundary
* rarely used in practice

Radial Basis Function (RBF) = commonly used kernel in SVC
2 parameters:
* gamma
* C

2. Gamma:
*  'spread' of the kernel and therefore the decision region.
* low gamma -> the 'curve' of the decision boundary is very low and thus the decision region is very broad (underfitting)
* gamma = 10 (The decision boundary starts to be highly effected by individual data points (i.e. variance)).
* high gamma -> the 'curve' of the decision boundary is high, which creates islands of decision-boundaries around data points (overfitting)
* If gamma is ‘auto’ then 1/n_features will be used instead.
* gamma defines how much influence a single training example has. 
* The larger gamma is, the closer other examples must be to be affected.

3. C:
* trades off misclassification of training examples against simplicity of the decision surface
* A low C makes the decision surface smooth, while a high C aims at classifying all training examples correctly
* penalty for misclassifying a data point
* small C -> classifier is okay with misclassified data points (high bias, low variance)
* big C -> classifier is heavily penalized for misclassified data and therefore bends over backwards avoid any misclassified data points (low bias, high variance)

C > 10 is too slow

4. degree : int, optional (default=3)
* Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.

In [135]:
#takes FOREVER
#linear 0.78
#rbf 0.76
#sigmoid 0.5

from sklearn.svm import SVC

kernels = ['linear', 'rbf', 'sigmoid']

for i in kernels:
    #for j, C in enumerate((0.01, 1, 10, 100)):
        #for k, D in enumerate((1, 10)):
    SVM = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
              decision_function_shape='ovr', degree=3, gamma='auto', kernel=i,
              max_iter=-1, probability=False, random_state=None, shrinking=True,
              tol=0.001, verbose=False)
    #SVM.fit(X_train, Y_train) 
    scores = cross_val_score(SVM, X_train, Y_train, cv = 10)

    #print ("Kernel: %s | Gamma: %0.2f | C: %i" % (i, C, D))
    print ("Kernel: %s" % (i))
    print scores
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Kernel: linear
[ 0.825       0.8125      0.725       0.7375      0.7721519   0.82278481
  0.83544304  0.69620253  0.76923077  0.79220779]
Accuracy: 0.78 (+/- 0.09)
Kernel: rbf
[ 0.7625      0.8375      0.7375      0.6875      0.72151899  0.70886076
  0.83544304  0.70886076  0.79487179  0.77922078]
Accuracy: 0.76 (+/- 0.10)
Kernel: sigmoid
[ 0.575       0.5125      0.4375      0.4375      0.49367089  0.48101266
  0.51898734  0.44303797  0.52564103  0.5974026 ]
Accuracy: 0.50 (+/- 0.11)


# 5. KNN

In [136]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', 
                           leaf_size=30, p=2, metric='minkowski', metric_params=None, 
                           n_jobs=1)
#KNN.fit(X_train, Y_train)
scores = cross_val_score(KNN, X_train, Y_train, cv = 10)

print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.8         0.7875      0.675       0.65        0.67088608  0.7721519
  0.79746835  0.7721519   0.75641026  0.75324675]
Accuracy: 0.74 (+/- 0.11)


## Features:

1. weights : str or callable, optional (default = ‘uniform’)
weight function used in prediction. Possible values:


* uniform : uniform weights. All points in each neighborhood are weighted equally.

* distance : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.

## II. Model Comparison

## 1. ROC Curves

In [ ]:
Adjusted R Square

ROC curve

auc Value

In [145]:
names = ["Decision Tree", "Neural Network", "Boosting", "SVM", "KNN"]

#implement ROC Curves
#

h = .02 

classifiers = [DT, NN, RF, SVM, KNN]